#Natural Language Processing for the Chinese Language -- Text Analysis of News Articles for Stock Price Prediction
End result:
1. Calculated sentiments for titles, article previews, and texts.
2. Used part-of-speech tagging to extract grammaticle components in a sentence, e.g. noun phrases, verbs etc, and ranked them based on frequency. 

Package Used 
*   SnowNLP
*   baiduAip


3. Used pointwise mutual information (algorithm) to rank words based on their positivity/negativity and frequency. 

In [ ]:
!pip install snownlp==0.12.3
!pip install nltk
!pip install stopwords-zh
!pip install baidu-aip

     |████████████████████████████████| 37.6MB 119kB/s 
  Created wheel for snownlp: filename=snownlp-0.12.3-cp36-none-any.whl size=37760959 sha256=3a8aeb4d7482b5bfe44cd3a9aa1334a10b95c9670699d6aaac858a89f68a8277
  Stored in directory: /root/.cache/pip/wheels/f3/81/25/7c197493bd7daf177016f1a951c5c3a53b1c7e9339fd11ec8f
Successfully built snownlp
ERROR: Could not find a version that satisfies the requirement stopwords-zh (from versions: none)
ERROR: No matching distribution found for stopwords-zh
  Created wheel for baidu-aip: filename=baidu_aip-2.2.18.0-cp36-none-any.whl size=15655 sha256=cd73974139d51bf9f67665a7cb346244df1705d69ccde00134f7d3a792049bde
  Stored in directory: /root/.cache/pip/wheels/5e/f3/20/9567d96b1140f13546bb3e059827cba0d575e213e8ee87f5ea
Successfully built baidu-aip


In [ ]:
import csv

import pandas as pd
import numpy as np
import math
from statistics import mean

from snownlp import SnowNLP 
from aip import AipNlp

from nltk.corpus import stopwords
import nltk
nltk.download("stopwords")

import codecs
import jieba
import jieba.analyse
import jieba.posseg as pseg

#language processing
import re
from bs4 import BeautifulSoup
import html

from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


#Pipeline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
StockNumber = '11_002057_中钢天源'
data = pd.read_csv('/content/drive/MyDrive/SZ data Nov 14/11_002057_中钢天源.csv')
data['id'] = StockNumber #file name

In [ ]:
#remove stopwords
stopwords = codecs.open('/content/drive/MyDrive/SZ data Nov 14/stopwords-zh.txt', 'r', 'utf-8').read().split('\n')
stopwordsList = ['分页']
# p = ['【','*','】','丨','(',')','，','%','-','”','“','、','）','（','：','；',',',':','/','+','=','\\',']','[','√','□','—','。',]
stopwords.extend(stopwordsList)
def removeStop(x):
  segList = SnowNLP(x).words
  # segList =[seg for seg in segList if seg not in stopwords]
  c = ''.join([s for s in segList if s.isalnum() and (s not in stopwords)])
  # print(segList)
  return c

def removeHtml(txt):
  # txt = re.sub(r'\\x(?=\w+)\w+',"",txt ) #anything after \\x 
  txt = re.sub(r'\S+page\S+',"",txt) 
  # for s in p:
  #   txt = re.sub('\\'+s, '',txt) #\[ match special characters

  # remove html tags
  soup = BeautifulSoup(txt[2:], "html.parser")
  txt = ''.join(soup.findAll(text=True))
  #Replace the HTML character codes with ASCII equvivalent  
  txt = html.unescape(txt)

  txt = re.sub(r"http\S+", "", txt) 
  txt = txt.strip()

  return txt

Data = data.copy()
Data['标题'] = Data['标题'].apply(lambda x: removeHtml(x))
Data['cfontnormal'] = Data['cfontnormal'].apply(lambda x: removeHtml(x))
Data['正文'] = Data.loc[Data['正文'].notnull(),'正文'].apply(lambda x: removeHtml(x))

Data = Data.replace('',np.NaN)

Data['标题'] = Data['标题'].apply(lambda x: removeStop(x))
Data['cfontnormal'] = Data['cfontnormal'].apply(lambda x: removeStop(x))
Data['正文'] = Data.loc[Data['正文'].notnull(),'正文'].apply(lambda x: removeStop(x))

In [ ]:
dataSnow = Data.copy()
def sentiSnow(x):
  senti = SnowNLP(x).sentiments
  if senti>0.5:
    return 1
  else:
    return 0

dataSnow['标题'] = dataSnow['标题'].apply(lambda x: sentiSnow(x))
dataSnow['cfontnormal'] = dataSnow['cfontnormal'].apply(lambda x: sentiSnow(x))
dataSnow['正文'] = dataSnow.loc[(dataSnow['正文'].notnull())&(dataSnow['正文']!=''),'正文'].apply(lambda x: sentiSnow(x))

In [ ]:
APP_ID = '22766485'
API_KEY = 'pTkeXFTmQSrVua41zcXTu0qi'
SECRET_KEY = 'dEFLyhVDQPcX63mDWeYHDpRm2xyVh0NX'

Appid = '22992872'
API_Key = 'q04yxeMPkqbTSR4mlgmSvx41'
Secret_Key = 'fUM3p1W89nEYBdcKPi6hWrCxV8QGr0kN'

client = AipNlp(Appid, API_Key, Secret_Key)

#maximal length for baidu api to process is 1000 words
# transform language into sentiment
dataAip = Data.copy()
# dataAip['标题'] = dataAip['标题'].apply(lambda x: client.sentimentClassify(x)['items'][0]['sentiment'])
# dataAip['cfontnormal'] = dataAip['cfontnormal'].apply(lambda x: client.sentimentClassify(x)['items'][0]['sentiment'])

def sentiCal(x,length):
  errorList = []
  try: 
    if len(x) < length:
      # print(x, client.sentimentClassify(x))
      senti = client.sentimentClassify(x)['items'][0]['sentiment']
      return senti 
    else:
      n = len(x)//length
      remindar = len(x)%length
      sentiList = []
      for a in range(n):
        senti = client.sentimentClassify(x[a:length+a])['items'][0]['sentiment']
        sentiList.append(senti)
      sentiList.append(client.sentimentClassify(x[-remindar:])['items'][0]['sentiment'])

      return sum(sentiList[:-1])*(n*length/len(x)) + sentiList[-1]*(remindar/len(x))
  except:
    errorList.append(x)

dataAip['标题'] = dataAip.loc[dataAip['标题'].notnull(),'标题'].apply(lambda x: sentiCal(x,1000))
dataAip['cfontnormal'] = dataAip.loc[dataAip['cfontnormal'].notnull(),'cfontnormal'].apply(lambda x: sentiCal(x,1000))
dataAip['正文'] = dataAip.loc[dataAip['正文'].notnull(),'正文'].apply(lambda x: sentiCal(x,1000))

dataAip.loc[dataAip['标题']>=1,'标题']=1
dataAip.loc[dataAip['标题']<1,'标题']=0
dataAip.loc[dataAip['cfontnormal']>=1,'cfontnormal']=1
dataAip.loc[dataAip['cfontnormal']<1,'cfontnormal']=0
dataAip.loc[dataAip['正文']>=1,'正文']=1
dataAip.loc[dataAip['正文']<1,'正文']=0

In [ ]:
jieba.enable_paddle()# 启动paddle模式。 0.40版之后开始支持，早期版本不支持

def getTopK(df, k1=30, k2=10,k3=50,col='正文'):
  counterDict = {}
  counterDictGram = {}
  Grammar = {}
  for Company in Data['id'].unique(): #标题1 will be replaced by identifier stock number
    mainText = df[(df['id']==Company)&(df[col].notnull())][col].values
    # counter = Counter()
    counter = []
    grammar = {}
    for m in mainText:
      for word,tag in pseg.cut(m):#jieba.cut(m,use_paddle=True)
        # print(word)
        counter.append(word)
        # counter.update(word)
        if tag not in grammar: 
          grammar[tag] = [word]
        else:
          grammar[tag].append(word)
        
        if tag not in Grammar:
          Grammar[tag] = [word]
        else:
          Grammar[tag].append(word)
    
    for k,v in grammar.items():
      grammar[k] = Counter(grammar[k]).most_common(k2)
    for k,v in Grammar.items():
      Grammar[k] = Counter(Grammar[k]).most_common(k2)
    counterDict[Company] = Counter(counter).most_common(k1)
    counterDictGram[Company] = grammar

  return counterDict, counterDictGram, Grammar

Paddle enabled successfully......


In [ ]:
#This function is to which terms occur the most frequently in the articles 
def getTopKF(df1, df2, col='正文',k1=50, k2=20): #df1=Data, df2=dataSnow/dataAip
  mainText = df1[col].values

  positiveCounter = []
  negativeCounter = []
  companyCounter = {} #for each company, create a dictionary

  for i in range(len(mainText)):
    if df2[col][i] == 1:
      # print(mainText[i])
      # print(i)
      positiveCounter.extend([word for word,tag in pseg.cut(mainText[i])])
    elif df2[col][i] == 0:
      negativeCounter.extend([word for word,tag in pseg.cut(mainText[i])])
  
  topkPositive = Counter(positiveCounter).most_common(k1)
  topkNegative = Counter(negativeCounter).most_common(k1)

  for Company in df1['id'].unique():
    mainText2 = df1[df1['id']==Company][col].values
    senti = {}
    senti['positive'] = []
    senti['negative'] = []
    for i in range(len(mainText2)):
      # print(df2[df2['id']==Company][col][i])
      if df2[df2['id']==Company][col].values[i] == 1:
        senti['positive'].extend([word for word,tag in pseg.cut(mainText2[i])])
      elif df2[df2['id']==Company][col].values[i] == 0:
        senti['negative'].extend([word for word,tag in pseg.cut(mainText2[i])])
    
    senti['positive'] = Counter(senti['positive']).most_common(k2)
    senti['negative'] = Counter(senti['negative']).most_common(k2)
    companyCounter[Company] = senti
    
  return topkPositive, topkNegative, companyCounter

In [ ]:
def dataFrameTransformation(counterDic, df1, df2, company,col='正文'): #df1 = Data, df2=dataSnow/dataAip
  #Find out if a particular review has the word from topk list
  freqReview = []
  reviews = df1[col].values
  for i in range(len(reviews)):
    # tempCounter = Counter([word for word,tag in pseg.cut(reviews[i]) if (df[col].values[i] = 1) or (df[col].values[i] = 0)])
    topkinReview = [1 if ((df2[col].values[i]==0) or  (df2[col].values[i]==1)) and (word in reviews[i])\
                    else 0 for (word, wordcount) in counterDic[company]]
    freqReview.append(topkinReview)

  freqReviewDf = pd.DataFrame(freqReview)
  dfName = []
  for c in counterDic[company]:
    dfName.append(c[0])
  freqReviewDf.columns = dfName

  finaldf = df2[[col,'id']].join(freqReviewDf)

  return freqReviewDf, finaldf

# Simple example of getting pairwise mutual information of a term
def pmiCal(df, x, col='正文'):
    pmilist=[]
    for i in [1,0]: #positive, negativ
        for j in [0,1]: #the word in the review or not
            px = sum(df[col]==i)/len(df)  #context: number of 'positive'/ all rows 
            py = sum(df[x]==j)/len(df) #word: number of reviews that conatain the word/ all rows 
            pxy = len(df[(df[col]==i) & (df[x]==j)])/len(df)
            # print(px,py,x)
            if pxy==0:#Log 0 cannot happen
                pmi = math.log((pxy+0.0001)/(px*py))
            else:
                pmi = math.log(pxy/(px*py))
            pmilist.append([i]+[j]+[px]+[py]+[pxy]+[pmi])
    pmidf = pd.DataFrame(pmilist)
    pmidf.columns = ['x','y','px','py','pxy','pmi']
    return pmidf

def pmiIndivCal(df,term,gt, col='正文'):
    px = sum(df[col]==gt)/len(df) #context: number of 'positive'/ all rows 
    py = sum(df[term]==1)/len(df) #word: number of reviews that conatain the word/ all rows 
    pxy = len(df[(df[col]==gt) & (df[term]==1)])/len(df)
    if pxy==0:#Log 0 cannot happen
        pmi = math.log((pxy+0.0001)/(px*py))
    else:
        pmi = math.log(pxy/(px*py))
    return pmi

def pmi(finaldf, counterDic, company, col='正文'):
    pmilist = []
    pmiposlist = []
    pmineglist = []
    for word in counterDic[company]:
        pmilist.append([word[0]]+[pmiCal(finaldf,word[0])]) #k *([the word, 'x','y','px','py','pxy','pmi'])
        pmiposlist.append([word[0]]+[pmiIndivCal(finaldf,word[0],1,col)]) #positive #k*([the word, pmi])
        pmineglist.append([word[0]]+[pmiIndivCal(finaldf,word[0],0,col)]) #negative #k*([the word, pmi])
    pmidf = pd.DataFrame(pmilist)
    pmiposlist = pd.DataFrame(pmiposlist)
    pmineglist = pd.DataFrame(pmineglist)
    pmiposlist.columns = ['word','pmi']
    pmineglist.columns = ['word','pmi']
    pmidf.columns = ['word','pmi']
    return pmiposlist, pmineglist, pmidf

In [ ]:
def split(x):
  x = ' '.join([word for word,tag in pseg.cut(x)])
  return x 

def tfidf(Data, TermList, company, col='正文'):
  vocabulary = list(TermList['word'])
  vectorizer = TfidfVectorizer(vocabulary=vocabulary, use_idf=True, smooth_idf=True, norm=None)
  Data = Data[(Data['id']==company)&(Data[col].notnull())]
  Data[col] = Data[col].apply(split)
  arrayTFIDF = vectorizer.fit_transform(Data[col]).toarray()
  dfTFIDF = pd.DataFrame(arrayTFIDF,columns=vocabulary)

  return dfTFIDF

In [ ]:
#Extract the most frequently mentioned terms and understand its correlation with sentiments

counterDic, counterDicGram, Grammar = getTopK(Data, k1=100)

#SnowNLP
# topkPositiveSnow, topkNegativeSnow, sentiCompanySnow = getTopKF(Data,dataSnow)
#Baidu Aip
topkPositiveB, topkNegativeB, sentiCompanyB = getTopKF(Data,dataAip)

#SnowNLP
# freqReviewDfSnow, finaldfSnow = dataFrameTransformation(counterDic, Data, dataSnow, StockNumber ,col='正文')
# pmiposlistSnow, pmineglistSnow, pmidfPosiSnow = pmi(finaldfSnow, counterDic, StockNumber)

#Baidu Aip
freqReviewDfAip, finaldfAip = dataFrameTransformation(counterDic, Data, dataAip, StockNumber ,col='正文')
pmiposlistAip, pmineglistAip, pmidfPosiAip = pmi(finaldfAip, counterDic, StockNumber)

#SnowNLP Positive/Negative terms 
# PosTermSnow = pmiposlistSnow.sort_values('pmi',ascending=0).head(50)
# NegTermSnow = pmineglistSnow.sort_values('pmi',ascending=False).head(50)
#BaiduAip
PosTermAip = pmiposlistAip.sort_values('pmi',ascending=0).head(50)
NegTermAip = pmineglistAip.sort_values('pmi',ascending=False).head(50)

#SnowNLP TFIDF
# dfTFIDF_Pos_Snow = tfidf(Data, PosTermSnow, StockNumber,'正文')
# dfTFIDF_Neg_Snow = tfidf(Data, NegTermSnow, StockNumber,'正文')
# BaiduAip
dfTFIDF_Pos_Aip = tfidf(Data, PosTermAip, StockNumber,'正文')
dfTFIDF_Neg_Aip = tfidf(Data, NegTermAip, StockNumber,'正文')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


#Analysis

##Differences between SnowNLP and Baidu

#Sentiment

In [ ]:
# filenameSnow = '/content/drive/My Drive/MarkThesis/Sentiment/'+ StockNumber+'SnowNLP_Sentiment.csv'
# dataSnow.to_csv(filenameSnow)

# filenameBaidu = '/content/drive/My Drive/MarkThesis/Sentiment/'+ StockNumber+'SnowNLP_Sentiment.csv'
# dataAip.to_csv(filenameBaidu)

In [ ]:
# filenameTFIDF_P_S = '/content/drive/My Drive/MarkThesis/TFIDF/'+ StockNumber+'_Positive_SnowNLP.csv'
# dfTFIDF_Pos_Snow.to_csv(filenameTFIDF_P_S )

# filenameTFIDF_N_S = '/content/drive/My Drive/MarkThesis/TFIDF/'+ StockNumber+'_Negative_SnowNLP.csv'
# dfTFIDF_Neg_Snow.to_csv(filenameTFIDF_N_S)


filenameTFIDF_P_B = '/content/drive/My Drive/MarkThesis/TFIDF_new/'+ StockNumber+'_Positive_BaiduAip.csv'
dfTFIDF_Pos_Aip.to_csv(filenameTFIDF_P_B)

filenameTFIDF_N_B = '/content/drive/My Drive/MarkThesis/TFIDF_new/'+ StockNumber+'_Negative_BaiduAip.csv'
dfTFIDF_Neg_Aip.to_csv(filenameTFIDF_N_B)